In [6]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

import os
import sys 
sys.path.append(os.path.abspath("../../../src"))   
import soporte_preprocesamiento as f
import warnings
warnings.filterwarnings("ignore")




# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor



from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, RobustScaler


from tqdm import tqdm


# import plotly_express as px


# Métodos estadísticos
# -----------------------------------------------------------------------
from scipy.stats import zscore # para calcular el z-score
from sklearn.neighbors import LocalOutlierFactor # para detectar outliers usando el método LOF
from sklearn.ensemble import IsolationForest # para detectar outliers usando el metodo IF
from sklearn.neighbors import NearestNeighbors # para calcular la epsilon

# Para generar combinaciones de listas
# -----------------------------------------------------------------------
from itertools import product, combinations
pd.set_option("display.max_columns", None)


In [7]:
df_escaldo=pd.read_csv("..\..\..\datos\datos0\datos_clusterizados0_escalado.csv", index_col=0)


In [11]:
# df_escaldo.head(2)

<!-- Como ya vimos, nuestras columnas numéricas eran Sales_unitarios y Discount y solo Sales_unitarios cuenta con outliers por ello al ser la variable respuesta habrá que tener cuidado -->

<!-- Dado que no tienen una distribución normal ninguna de ellas, no utilizaré el Z-Score ya que el porcentaje de fallo será demasiado alto. Por ello opto por el IQR -->

In [12]:
# dicc_iqr=f.identificar_outliers_iqr(df_escaldo)

<!-- Nos fijamos en que parece haber muy pocos Outliers -->

In [13]:
# dicc_iqr["Sales_unitario"].head(2)

<!-- El número de outliers no es muy elevado (entorno al 6%) pero al tener sentido como ya se observaron en pasos anteirores voy a optar por no tocarlos  -->

In [14]:
# contaminacion= [0.01, 0.05, 0.1]
# estimadores = [100,400,1000, 2000] 
# combinaciones= list(product(contaminacion, estimadores))
# for cont, esti in combinaciones:
    
#     ifo=IsolationForest(random_state=42, n_estimators=esti, contamination= cont, n_jobs=-1)         #n_estimator es el número de árboles y n_jobs con -1 coge todos los nucleos del ordenador

#     df_escaldo[f"outliers_ifo_{cont}_{esti}"]=ifo.fit_predict(df_escaldo[["price","size_MinMax","distance_MinMax"]]) 

In [15]:
# df_min["outliers_ifo_0.01_100"].value_counts(normalize=True)    #Esto nos muetsra como se acerca mucho al 1% de outliers por el nivel de contaminación ser 0.01

In [16]:
# df_min.head()

In [17]:
# columnas_hue=df_min.filter(like="outliers")

In [18]:
# df_min.head()

<!-- 
A pesar de que teóricamente el IFO por las carcaterísticas de los datos parece ser la mejor opción y tras haber observado que marca outliers en los precios ma´s bajo. Pasamos ahora a identificar los outliers con la metodología LOF.

Para poder comparar más fácil voy a sacar IFO y LOF juntos. -->

In [19]:
# lista_contaminaciones = [0.01, 0.05, 0.1]
# lista_neighbors = [8,20, 50]

# combinaciones = list(product(lista_contaminaciones, lista_neighbors))
# combinaciones


# for cont, neighbors in tqdm(combinaciones):
#     lof = LocalOutlierFactor(n_neighbors=neighbors,
#                             contamination=cont,
#                             n_jobs=-1)

#     df_min[f"outliers_lof_{cont}_{neighbors}"] = lof.fit_predict(df_min[["price","size_MinMax","distance_MinMax"]])
#     y_pred = lof.fit_predict(df_min[["price","size_MinMax","distance_MinMax"]])
# df_min

# # #visualizacion
# # columnas_hue = df_min.filter(like="outlier").columns

# # combinaciones_viz = list(combinations(["price","size_MinMax","distance_MinMax"], 2))
# # combinaciones_viz

# # for outlier in tqdm(columnas_hue):
# #     fig, axes = plt.subplots(nrows=1, ncols=3, figsize = (15, 5))
# #     axes = axes.flat

# #     for indice, tupla in enumerate(combinaciones_viz):
# #         sns.scatterplot(x = tupla[0],
# #                         y = tupla[1],
# #                         ax = axes[indice],
# #                         data = df_min,
# #                         hue=outlier,
# #                         palette="Set1",
# #                         style=outlier,
# #                         alpha=0.5)
        
# #     plt.suptitle(outlier)

<!-- Lo que saco en claro es que el nivel de contaminación más estricto (10%) es la mejor opción porque es el que más se acerca a la realidad de los datos.  

Tras observar los gráficos del LOF observo que en el caso del tamaño, se consideran outliers aquellas viviendas que cuentan con un alto precio y según van aumentando su tamaño se va reduciendo el número de outliers. Por otro lado, al  observar la distancia nos percatamos de que de nuevo los outliers se concentran en los precios más altos lo que en un principio parece no tener sentido pues se mantienen aproximadamente igual distribuidos a lo largo de las distintas distancias. Sin embargo, no nos podemos fiar porque puede que esos datos sean outliers por otras variables que no sea la distancia.

Por el momento, me parece más adecuado hacer uso de la metodología IFO. -->


In [20]:
# df_min.columns

In [21]:
# df_ifo=df_min[['price', 'propertyType', 'exterior', 'rooms', 'bathrooms', 'province',
#        'municipality', 'status', 'newDevelopment', 'floor', 'district',
#        'hasLift', 'parkingSpace', 'size_MinMax', 'distance_MinMax',
#        'outliers_ifo_0.01_100', 'outliers_ifo_0.01_400',
#        'outliers_ifo_0.01_1000', 'outliers_ifo_0.01_2000',
#        'outliers_ifo_0.05_100', 'outliers_ifo_0.05_400',
#        'outliers_ifo_0.05_1000', 'outliers_ifo_0.05_2000',
#        'outliers_ifo_0.1_100', 'outliers_ifo_0.1_400', 'outliers_ifo_0.1_1000',
#        'outliers_ifo_0.1_2000']]

In [22]:
# df_ifo.head()

<!-- Ahora me quito las que tengan todas las filas con 1 y así me quedo con todas las que al menos tienen un Outlier -->

In [23]:
# columnasdf=df_min.filter(like="outliers_ifo")
# columnas_ifo=columnasdf.columns

In [24]:

# filtered_df = df_ifo[(df_ifo[columnas_ifo] == -1).all(axis=1)]

In [25]:
# filtered_df

In [26]:
# filtered_df.shape

<!-- Al observar las filas que tienen todos los valores con -1 y observar que suponen una parte ínfima de los datos, se decide eliminarlos -->

In [27]:
# df_result = df_ifo.drop(index = filtered_df.index)

In [28]:
# df_result.reset_index(drop=True, inplace=True)

In [29]:
# df_result.shape

<!-- Ahora seleccionamos el 60% de los outliers -->

In [30]:
# proporcion = 0.6 * len(columnas_ifo)
# df_outliers_60 = df_result[df_result[columnas_ifo].eq(-1).sum(axis=1) >= proporcion]


In [31]:
# df_ifo.shape

In [32]:
# df_outliers_60.shape

<!-- Observamos la distancia -->

In [33]:
# df_outliers_60["distance_MinMax"].shape

In [34]:
# df_outliers_60[df_outliers_60["distance_MinMax"] > 0.9]["distance_MinMax"]

In [35]:
# df_outliers_60[df_outliers_60["distance_MinMax"] > 0.9]["distance_MinMax"].shape

In [36]:
# df_outliers_60[df_outliers_60["distance_MinMax"] > 0]["distance_MinMax"].unique()

<!-- Estamos observando los valores mayores a 0.9 (aproximadamente a partir de 55 kilometros del centro) -->

In [37]:
# df_outliers_60.info()

In [38]:
# df_outliers_60["distance_MinMax"].min()

In [39]:
# df_outliers_60["distance_MinMax"].max()

In [40]:
# df_outliers_60[df_outliers_60["distance_MinMax"] > 0].shape

In [41]:
# df_outliers_60["distance_MinMax"].shape

<!-- Aqui quiero crear el intervalo de outliers con el que me voy a quedar.  
Dado que las viviendas con un valor de distancia mayor a 0.9 (unos 55km aproximadamente del centro) me las marca por lo general como outliers y dado que no hay muchas se eliminan, para el resto se tratará de imputar su valor. -->

In [42]:
# df_outliers_60=df_outliers_60[(df_outliers_60["distance_MinMax"] > 0 ) & (df_outliers_60["distance_MinMax"] < 0.9)]

<!-- Casi la mitad de otliers se encuentra entre 0.9 y 1 (55 y 60 kilometros de distancia aproximadamente). Esto muestra que hay otra mitad de viviendas (outliers) entre los 0 km y los 55 km aproximadamente cosa que tiene sentido con lo que no puedo eliminar estos outliers.El resto si que no tiene sentido mantenerlos (>0.9) -->

<!-- Ahora observamos los tamaños -->

In [43]:
# df_outliers_60["size_MinMax"].shape

In [44]:
# df_outliers_60["size_MinMax"].unique()

<!-- 
En los outliers de size: media(0.47) y mediana (0.39) indica que en los outliers hay valores atípicos que distorsionan la media (Ej:1 o 0.85271318)  
En los outliers de distance: media(0.52) y mediana (0.57) indica que en los outliers hay valores atípicos que distorsionan la media (Ej:0.00739922 o 0.00980983)   -->

In [45]:
# pd.options.display.float_format = "{:,.2f}".format 
# df_outliers_60.describe().T

In [46]:
# df_outliers_60[df_outliers_60["size_MinMax"] < 0.35]["size_MinMax"].unique()

In [47]:
# df_outliers_60[df_outliers_60["size_MinMax"] < 0.35].shape

In [48]:
# df_outliers_60[df_outliers_60["size_MinMax"] < 0.35]

In [49]:
# df_outliers_60[df_outliers_60["size_MinMax"] > 0.35].shape

In [50]:
# df_outliers_60[df_outliers_60["size_MinMax"] > 0.35]

<!-- Aqui quiero crear el intervalo de outliers con el que me voy a quedar de size  
Dado que las viviendas con un valor de size menor a 0.35 (75 metros cuadrados aproximadamente) y mayor a 0.84 (135 metros cuadrados aproximadamente) me las marca por lo general como outliers y dado que no hay muchas se eliminan, para el resto se tratará de imputar su valor. -->

In [51]:
# df_outliers_60=df_outliers_60[(df_outliers_60["size_MinMax"] > 0.35 ) & (df_outliers_60["size_MinMax"] < 0.84)]

In [52]:
# df_outliers_60.shape

<!-- Ahora a que ya hemos analizado los outliers decidimos imputarlos usando KNN con lo que primero pasamos todos a nan y los meto en el dataframe completo donde voy a querer meter el imputer KNN (y probaremos con el iterative también) -->

In [53]:
# df_result.info()

In [54]:
# df_result.loc[df_outliers_60.index, "size_MinMax"] = np.nan
# df_result.loc[df_outliers_60.index, "distance_MinMax"] = np.nan

In [55]:
# df_result.info()

In [56]:
# df_numericas=df_result.select_dtypes(include=np.number)
# imputer_knn= KNNImputer(n_neighbors=5)   #por defecto siempre lo calcula en base a la media y no se puede cambiar
# knn_imputado= imputer_knn.fit_transform(df_numericas)    #como es solo para numericas se tiene que meter el df de numericas pero hay que definirlo otra vez porque hemos dropeado duplicados
# df_knn=df_result.copy()
# df_num_sin_nulos=pd.DataFrame(knn_imputado, columns= df_numericas.columns)  #para que tenga los mismo nombres de columnas
# df_knn[df_numericas.columns]= df_num_sin_nulos    #Con esto metemos todo el dataframe de las numericas sin nulos donde corresponda rellenando esos nulos


In [57]:
# df_knn.isna().sum()

In [58]:
# df_knn.describe().T

In [59]:
# df_knn.to_csv("../../datos/datos1/datos_sin_outliers.csv")

In [60]:
# df=pd.read_csv("../../datos/datos1/datos_sin_outliers.csv", index_col=0).reset_index(drop=True)

In [61]:
# df_filt=df[["price", "size_MinMax", "distance_MinMax"]]

In [62]:
# f.plot_outliers_univariados(df_filt, "b", (15,10))

In [63]:
# df_est_con3=df_min[["price", "size_MinMax", "distance_MinMax"]]

In [64]:
# (df_filt["size_MinMax"]> 0.8).sum()

In [65]:
# (df_est_con3["size_MinMax"]> 0.8).sum()

In [66]:
# (df_filt["price"]< 510).sum()

In [67]:
# (df_est_con3["price"]< 510).sum()

In [68]:
# f.plot_outliers_univariados(df_est_con3, "b", (15,10))

<!-- caso con iterative -->

In [69]:
# df_numericas=df_result.select_dtypes(include=np.number)
# df_numericas.drop(columns="price", inplace=True)
# imputer_iterative= IterativeImputer()       #Aquí se puede poner el KNNImputer o el RandomForestRegressor
#                                             #missing values es por si los nulos no estan como np.nan sino en texto por ejemplo (aunque lo suyo es limpiarlo en el EDA)
#                                             #initial_strategy= "median" sirve para indicar que me lo haga con las medianas
# iterative_imputado=imputer_iterative.fit_transform(df_numericas)   #como es solo para numericas se tiene que meter el df de numericas pero hay que definirlo otra vez porque hemos dropeado duplicados
# df_iterativo=df_result.copy()
# df_num_sin_nulos=pd.DataFrame(iterative_imputado, columns= df_numericas.columns)  #para que tenga los mismo nombres de columnas
# df_iterativo[df_numericas.columns]= df_num_sin_nulos    #Con esto metemos todo el dataframe de las numericas sin nulos donde corresponda rellenando esos nulos

In [70]:
# df_iterativo.isna().sum()

In [71]:
# df_iterativo.describe().T

In [72]:
# df_iterativo.to_csv("../../datos/datos1/datos_sin_outliers_iterativo.csv")

In [73]:
# df_it=pd.read_csv("../../datos/datos1/datos_sin_outliers_iterativo.csv", index_col=0).reset_index(drop=True)

In [74]:
# df_filt_it=df_it[["price", "size_MinMax", "distance_MinMax"]]

In [75]:
# f.plot_outliers_univariados(df_filt_it, "b", (15,10))

In [76]:
# df_est_con3=df_min[["price", "size_MinMax", "distance_MinMax"]]

In [77]:
# (df_filt_it["size_MinMax"]> 0.8).sum()

In [78]:
# (df_est_con3["size_MinMax"]> 0.8).sum()

In [79]:
# (df_filt_it["price"]< 510).sum()

In [80]:
# (df_est_con3["price"]< 510).sum()

In [81]:
# f.plot_outliers_univariados(df_est_con3, "b", (15,10))

<!-- Ya están tratados -->

<!-- Observamos que no hay diferencias muy significativas entre imputar con KNN o con ITERATIVE pero tras observar que el iterative reduce la dispersión entre la media y la mediana de size y que el KNN aumenta la desviación típica (variabilidad) en la distancia, nos decantamos finalmente por el uso del ITERATIVE -->